In [1]:
! pip install transformers datasets

In [2]:
from datasets import load_dataset

# Load HotpotQA dataset
hotpot_dataset = load_dataset("hotpot_qa", "distractor", trust_remote_code=True)

# Display HotpotQA dataset structure
print(hotpot_dataset)

Generating train split:   0%|          | 0/90447 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7405 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 90447
    })
    validation: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 7405
    })
})


In [3]:
# Load WikiNQ dataset
wiki_dataset = load_dataset("Tevatron/wikipedia-nq")

# Display WikiNQ dataset structure
print(wiki_dataset)

Generating train split:   0%|          | 0/58622 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/6489 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3610 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['query_id', 'query', 'answers', 'positive_passages', 'negative_passages'],
        num_rows: 58622
    })
    dev: Dataset({
        features: ['query_id', 'query', 'answers', 'positive_passages', 'negative_passages'],
        num_rows: 6489
    })
    test: Dataset({
        features: ['query_id', 'query', 'answers', 'positive_passages', 'negative_passages'],
        num_rows: 3610
    })
})


In [4]:
hotpot_train_set = hotpot_dataset['train']
hotpot_test_set = hotpot_dataset['validation']

wiki_train_set = wiki_dataset['train']
wiki_test_set = wiki_dataset['dev']

In [5]:
small_hotpot_train_dataset = hotpot_train_set.shuffle(seed=42).select(range(2000))
small_hotpot_test_dataset = hotpot_test_set.shuffle(seed=42).select(range(1000))

small_wiki_train_dataset = wiki_train_set.shuffle(seed=43).select(range(2000))
small_wiki_test_dataset = wiki_test_set.shuffle(seed=43).select(range(1000))

In [6]:
import random

def get_random_negatives(golden_doc_titles, corpus, num_negatives):
    """Randomly sample negatives excluding golden_docs."""
    available_doc_titles = [doc_title for doc_title in corpus if doc_title not in golden_doc_titles]
    sampled_titles = random.sample(available_doc_titles, min(len(available_doc_titles), num_negatives))
    negatives = [''.join(corpus[doc_title]) for doc_title in sampled_titles]
    return negatives

In [7]:
def create_corpus_hotpot(data, sample_method, batch_size):
    """Preprocess data to create datapoints with negatives."""
    if sample_method == 'global':
        global_corpus = {}
        for entry in data:
            for title, sentences in zip(entry['context']['title'], entry['context']['sentences']):
                if title not in global_corpus:
                    global_corpus[title] = ''.join(sentences)
        return global_corpus
    elif sample_method == 'inbatch':
        data_list = list(data)
        batches = [data_list[i:i + batch_size] for i in range(0, len(data_list), batch_size)]
        batch_corpuses = []
        for batch in batches:
            batch_corpus = {}
            for entry in batch:
                for title, sentences in zip(entry['context']['title'], entry['context']['sentences']):
                    if title not in batch_corpus:
                        batch_corpus[title] = ''.join(sentences)
            batch_corpuses.append(batch_corpus)
        return batch_corpuses
    else:
        assert(0)


In [8]:
# Preprocessing function Hotpot
def preprocess_data_train_hotpot(data, num_negatives, sample_method='global', batch_size=32):
    """Preprocess data to create datapoints with negatives."""
    corpuses = create_corpus_hotpot(data, sample_method, batch_size)

    processed_data = []

    for idx, entry in enumerate(data):
        query = entry['question']
        golden_doc_titles = entry['supporting_facts']['title']
        context = entry['context']
        if sample_method == 'global':
            corpus = corpuses
        else:
            corpus = corpuses[idx // batch_size]

        for golden_doc_title in golden_doc_titles:
            golden_doc = ''.join(context['sentences'][context['title'].index(golden_doc_title)])
            negatives = get_random_negatives(golden_doc_titles, corpus, num_negatives)

            datapoint = {
                'query': query,
                'doc': golden_doc,
                'negatives': negatives,
            }
            processed_data.append(datapoint)

    return processed_data

In [9]:
# Preprocessing function Hotpot
def preprocess_data_test_hotpot(data, num_negatives, sample_method='global', batch_size=32):
    """Preprocess data to create datapoints with negatives."""
    corpuses = create_corpus_hotpot(data, sample_method, batch_size)

    processed_data = []

    for idx, entry in enumerate(data):
        query = entry['question']
        golden_doc_titles = entry['supporting_facts']['title']
        context = entry['context']
        if sample_method == 'global':
            corpus = corpuses
        else:
            corpus = corpuses[idx // batch_size]

        positives = [''.join(context['sentences'][context['title'].index(golden_doc_title)]) for golden_doc_title in golden_doc_titles]
        negatives = get_random_negatives(golden_doc_titles, corpus, num_negatives * len(positives))

        datapoint = {
            'query': query,
            'positives': positives,
            'negatives': negatives,
        }
        processed_data.append(datapoint)

    return processed_data

In [10]:
# Preprocessing function WikiNQ
def preprocess_data_train_wiki(data, num_negatives):
    processed_data = []

    for idx, entry in enumerate(data):
        query = entry['query']
        golden_docs = entry['positive_passages']
        negative_docs = entry['negative_passages']

        for golden_doc in golden_docs:
            negatives = random.sample(negative_docs, min(len(negative_docs), num_negatives))
            negatives = [neg['text'] for neg in negatives]

            datapoint = {
                'query': query,
                'doc': golden_doc['text'],
                'negatives': negatives,
            }
            processed_data.append(datapoint)

    return processed_data

In [11]:
# Preprocessing function WikiNQ
def preprocess_data_test_wiki(data, num_negatives):
    processed_data = []

    for idx, entry in enumerate(data):
        query = entry['query']
        golden_docs = entry['positive_passages']
        negative_docs = entry['negative_passages']

        positives = [golden_doc['text'] for golden_doc in golden_docs]
        negatives = random.sample(negative_docs, min(len(negative_docs), num_negatives * len(positives)))
        negatives = [neg['text'] for neg in negatives]

        datapoint = {
            'query': query,
            'positives': positives,
            'negatives': negatives,
        }
        processed_data.append(datapoint)

    return processed_data

In [12]:
hotpot_train_data_random = preprocess_data_train_hotpot(small_hotpot_train_dataset, 7, 'global')[:2500]
hotpot_train_data_inbatch = preprocess_data_train_hotpot(small_hotpot_train_dataset, 7, 'inbatch', 16)[:2500]

hotpot_test_data_random = preprocess_data_test_hotpot(small_hotpot_test_dataset, 15, 'global')[:2500]
hotpot_test_data_inbatch = preprocess_data_test_hotpot(small_hotpot_test_dataset, 15, 'inbatch', 16)[:2500]

In [13]:
wiki_train_data = preprocess_data_train_wiki(small_wiki_train_dataset, 7)[:2500]

wiki_test_data = preprocess_data_test_wiki(small_wiki_test_dataset, 15)[:2500]

In [14]:
import torch
from transformers import BertTokenizer, BertForPreTraining

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

def tokenize_input(query, document):
    return tokenizer.encode_plus(
        query,
        document,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer

class DocumentLikelihoodModel(nn.Module):
    def __init__(self, bert_model_name='bert-base-cased', hidden_size=768, ffl_depth=1, dropout_prob=0.3):
        super(DocumentLikelihoodModel, self).__init__()

        # Load pre-trained BERT model
        self.bert = BertModel.from_pretrained(bert_model_name)

        # Define a feedforward layer stack (FFL) over the CLS token
        layers = []
        input_size = hidden_size
        for _ in range(ffl_depth):
            layers.append(nn.Linear(input_size, hidden_size))
            layers.append(nn.ReLU())                    # Non-linearity
            layers.append(nn.Dropout(dropout_prob))      # Dropout
            input_size = hidden_size

        self.ffl = nn.Sequential(*layers)

        # Final layer to output a single score
        self.output_layer = nn.Linear(hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        # Pass inputs through BERT
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)

        # Get the [CLS] token representation
        cls_output = outputs.last_hidden_state[:, 0, :]  # CLS token is the first one in the sequence

        # Pass CLS representation through the FFL
        ffl_output = self.ffl(cls_output)

        # Final linear layer to output a score
        score = self.output_layer(ffl_output).view(-1)
        return score

In [16]:
from torch.nn import functional as F

def compute_document_likelihood(model, encoding):
    # Forward pass to get the logits
    return model(input_ids=encoding['input_ids'], attention_mask=encoding['attention_mask'])

In [17]:
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

CELoss = CrossEntropyLoss()

def train_document_likelihood_model(train_data, model, optimizer, epochs=3, device='cuda'):
    model.to(device)
    model.train()

    for epoch in range(epochs):
        total_loss = 0

        for data in tqdm(train_data, desc=f"Epoch {epoch+1}", total=len(train_data)):
            query = data['query']
            positive_doc = data['doc']
            negatives = data['negatives']

            # Move inputs to device (e.g., GPU if available)
            encoding_pos = tokenize_input(query, positive_doc).to(device)
            encoding_negs = [tokenize_input(query, neg_doc).to(device) for neg_doc in negatives]

            # Compute likelihoods for the document and negatives
            sim_pos = compute_document_likelihood(model, encoding_pos)
            sim_negs = torch.stack([compute_document_likelihood(model, encoding_neg) for encoding_neg in encoding_negs])

            all_similarities = torch.cat([sim_pos.unsqueeze(0), sim_negs], dim=0).view(-1)
            target = torch.cat([torch.ones(1), torch.zeros(len(sim_negs))], dim=0).to(device)

            loss = CELoss(all_similarities, target)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_data)
        print(f'Epoch {epoch+1}, Average Loss: {avg_loss}')

In [18]:
def evaluate_document_likelihood_model(eval_data, model, device='cuda'):
    model.to(device)
    model.eval()

    precision_at_1 = 0
    precision_at_10 = 0
    mrr_total = 0
    map_total = 0
    total_queries = len(eval_data)

    with torch.no_grad():
        for data in tqdm(eval_data, desc="Evaluating", total=total_queries):
            query = data['query']
            positives = data['positives']
            negatives = data['negatives']

            # Move inputs to device (e.g., GPU if available)
            encoding_poses = [tokenize_input(query, pos_doc).to(device) for pos_doc in positives]
            encoding_negs = [tokenize_input(query, neg_doc).to(device) for neg_doc in negatives]

            # Compute likelihoods for the document and negatives
            sim_poses = torch.stack([compute_document_likelihood(model, encoding_pos) for encoding_pos in encoding_poses])
            sim_negs = torch.stack([compute_document_likelihood(model, encoding_neg) for encoding_neg in encoding_negs])

            # Concatenate positive and negative likelihoods
            all_similarities = torch.cat([sim_poses, sim_negs], dim=0).view(-1)

            # Get the ranking of the positive document (rank is based on the similarity score)
            rankings = torch.argsort(torch.argsort(all_similarities, descending=True))
            rank_of_positives = rankings[:len(sim_poses)]

            if 0 in rank_of_positives:
                precision_at_1 += 1
            precision_at_10 += len(list(filter(lambda x: x < 10, rank_of_positives))) / min(len(all_similarities), 10)

            # Compute Reciprocal Rank for MRR
            rank_of_first_positive = min(rank_of_positives) + 1
            mrr_total += 1 / rank_of_first_positive

            # Compute Average Precision for MAP
            ap = 0.0
            for i, rank in enumerate(sorted(rank_of_positives)):
                ap += (i + 1) / (rank + 1)  # Precision at each rank where a positive document appears
            map_total += ap / len(rank_of_positives)  # Average precision for this query

    # Average the metrics over all queries
    precision_at_1 /= total_queries
    precision_at_10 /= total_queries
    mrr_total /= total_queries
    map_total /= total_queries

    print(f'Precision@1: {precision_at_1}')
    print(f'Precision@10: {precision_at_10}')
    print(f'MRR: {mrr_total}')
    print(f'MAP: {map_total}')

In [19]:
import warnings
import logging
warnings.filterwarnings('ignore')
logging.getLogger("transformers").setLevel(logging.ERROR)

In [20]:
# Train with Random/Global Negatives Hotpot
model_hotpot_random = DocumentLikelihoodModel(bert_model_name='bert-base-cased', ffl_depth=1, dropout_prob=0.3)
# Optimizer
optimizer_hotpot_random = torch.optim.AdamW(model_hotpot_random.parameters(), lr=5e-5)

train_document_likelihood_model(hotpot_train_data_random, model_hotpot_random, optimizer_hotpot_random)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Epoch 1: 100%|██████████| 2500/2500 [34:42<00:00,  1.20it/s]


Epoch 1, Average Loss: 0.34429733961432113


Epoch 2: 100%|██████████| 2500/2500 [34:41<00:00,  1.20it/s]


Epoch 2, Average Loss: 0.10672015976096887


Epoch 3: 100%|██████████| 2500/2500 [34:39<00:00,  1.20it/s]

Epoch 3, Average Loss: 0.21297455701449716


In [21]:
evaluate_document_likelihood_model(hotpot_test_data_random, model_hotpot_random)

Evaluating: 100%|██████████| 1000/1000 [21:39<00:00,  1.30s/it]

Precision@1: 0.982

Precision@10: 0.24039999999999817

MRR: 0.9907000660896301

MAP: 0.9496010541915894


In [22]:
# Train with Inbatch Negatives Hotpot
model_hotpot_inbatch = DocumentLikelihoodModel(bert_model_name='bert-base-cased', ffl_depth=1, dropout_prob=0.3)
# Optimizer
optimizer_hotpot_inbatch = torch.optim.AdamW(model_hotpot_inbatch.parameters(), lr=5e-5)

train_document_likelihood_model(hotpot_train_data_inbatch, model_hotpot_inbatch, optimizer_hotpot_inbatch)

Epoch 1: 100%|██████████| 2500/2500 [33:47<00:00,  1.23it/s]


Epoch 1, Average Loss: 0.434711778548466


Epoch 2: 100%|██████████| 2500/2500 [33:47<00:00,  1.23it/s]


Epoch 2, Average Loss: 0.23303163344577318


Epoch 3: 100%|██████████| 2500/2500 [33:47<00:00,  1.23it/s]

Epoch 3, Average Loss: 0.1510773437827356


In [23]:
evaluate_document_likelihood_model(hotpot_test_data_inbatch, model_hotpot_inbatch)

Evaluating: 100%|██████████| 1000/1000 [14:22<00:00,  1.16it/s]

Precision@1: 0.895
Precision@10: 0.24109999999999812
MRR: 0.94159996509552
MAP: 0.8806471824645996


In [20]:
# Train with WikiNQ
model_wiki = DocumentLikelihoodModel(bert_model_name='bert-base-cased', ffl_depth=1, dropout_prob=0.3)
# Optimizer
optimizer_wiki = torch.optim.AdamW(model_wiki.parameters(), lr=5e-5)

train_document_likelihood_model(wiki_train_data, model_wiki, optimizer_wiki)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Epoch 1: 100%|██████████| 2500/2500 [33:46<00:00,  1.23it/s]


Epoch 1, Average Loss: 2.0066116576363333


Epoch 2: 100%|██████████| 2500/2500 [33:49<00:00,  1.23it/s]


Epoch 2, Average Loss: 2.0905015123844146


Epoch 3: 100%|██████████| 2500/2500 [33:46<00:00,  1.23it/s]

Epoch 3, Average Loss: 2.0900320539951323


In [21]:
evaluate_document_likelihood_model(wiki_test_data, model_wiki)

Evaluating: 100%|██████████| 1000/1000 [23:27<00:00,  1.41s/it]

Precision@1: 0.075
Precision@10: 0.0936999999999997
MRR: 0.20378009974956512
MAP: 0.16959945857524872


In [ ]:
# Train with Inbatch Negatives WikiNQ
model_wiki_inbatch = DocumentLikelihoodModel(bert_model_name='bert-base-cased', ffl_depth=1, dropout_prob=0.3)
# Optimizer
optimizer_wiki_inbatch = torch.optim.AdamW(model_wiki_inbatch.parameters(), lr=5e-5)

train_query_likelihood_model(wiki_train_data_inbatch, model_wiki_inbatch, optimizer_wiki_inbatch)

In [ ]:
evaluate_query_likelihood_model(wiki_test_data_inbatch, model_wiki_inbatch)